In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np

from keras.datasets import mnist
from keras.utils import to_categorical

import keras
from keras import layers
from keras import models
import tensorflow as tf

from tensorflow.keras.applications import VGG16

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

val_images = train_images[:6000]
val_labels = train_labels[:6000]

train_images = train_images[6000:]
train_labels = train_labels[6000:]


In [ ]:
def procesa_imagenes(dataset):
  new_dataset = []
  for img in dataset:
    img = cv.resize(img, (56, 56))
    img = cv.cvtColor (img, cv.COLOR_GRAY2BGR)
    new_dataset.append(img)
  return np.array(new_dataset)

In [ ]:
train_images = procesa_imagenes(train_images)
test_images = procesa_imagenes(test_images)
val_images = procesa_imagenes(val_images)

In [ ]:
train_images = train_images.astype("float32")/255
test_images = test_images.astype("float32")/255
val_images = val_images.astype("float32")/255

In [ ]:
train_images.shape

In [ ]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)
val_labels = to_categorical(val_labels)

In [ ]:
pre_trainded_model = VGG16(input_shape=(train_images.shape[1],train_images.shape[2],train_images.shape[3]),
                           weights = "imagenet",
                           include_top=False)

In [ ]:
pre_trainded_model.summary()

In [ ]:
pre_trainded_model

In [ ]:
for ls in pre_trainded_model.layers:
    ls.trainable = False

In [ ]:
pre_trainded_model.summary()

In [ ]:
model = tf.keras.models.Sequential([
    pre_trainded_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax"),
])
model.summary()

In [ ]:
model.compile(loss="categorical_crossentropy",optimizer="Adam",metrics=['Accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_images, train_labels,
                    batch_size=100,
                    epochs=1,
                    validation_data=(val_images, val_labels))

In [ ]:
import pandas as pd

# Cuando hay una separación notable tras cruzada es el sobreentrenamiento
pd.DataFrame(history.history).plot()

In [ ]:
mse_test = model.evaluate(test_images,test_labels)
#Modelo con los datos de Test para ver el rendimiento

In [ ]:
pre_trained_model = VGG16(input_shape=(56,56,3),
                          weights = "imagenet",
                          include_top = False)

pre_trained_model.trainable = True

trainable = False
for layer in pre_trained_model.layers:
  if layer.name == "block5_conv1":
    trainable = True
  layer.trainable = trainable

pre_trained_model.summary()

In [ ]:
history = model.fit(train_images, train_labels,
                    batch_size=100,
                    epochs=3,
                    validation_data=(val_images, val_labels))